<a href="https://colab.research.google.com/github/vkathole/plant-disease-detection/blob/main/vgg_trained_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split  

N_IMAGES=400
data_dir = '/content/drive/My Drive/data/train'
categories=['Tomato__Target_Spot','Tomato__Tomato_mosaic_virus','Tomato_Bacterial_spot',
            'Tomato_Early_blight','Tomato_healthy','Tomato_Late_blight','Tomato_Leaf_Mold']

In [ ]:
data=[]
def make_data():
    for category in categories:
        path=os.path.join(data_dir,category)
        label=categories.index(category)
        
        for img_name in os.listdir(path)[:N_IMAGES]:
            image_path=os.path.join(path,img_name)
            image=cv2.imread(image_path)
            
            try:
                image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image=cv2.resize(image,(224,224))
                
                image=np.array(image,dtype=np.float32)
                data.append([image,label])
            except Exception as e:
                pass
    pik=open('data.pickle','wb')
    pickle.dump(data,pik)
    pik.close()
    print(len(data))

make_data()

In [ ]:
def load_data():
    pick=open('data.pickle','rb')
    data=pickle.load(pick)
    pick.close()
    
    np.random.shuffle(data)
    
    feature=[]
    labels=[]
    
    for img,label in data:
        feature.append(img)
        labels.append(label)
    
    feature=np.array(feature,dtype=np.float32)
    labels=np.array(labels)
    feature=feature/255.0
    
    return[feature,labels]

load_data()

In [ ]:
from keras.applications.vgg16 import VGG16
model_vgg = VGG16(weights='imagenet', include_top=False, input_shape= (224,224,3))


In [ ]:
model_vgg.trainable = False

In [ ]:
model = keras.Sequential()

# Adding pre-trained model layers
model.add(model_vgg)

# Flattening the image pixels
model.add(Flatten())

# Adding 2 dense layers with 4096 neurons
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))

# Adding an output layer with 2 neurons
model.add(Dense(7, activation='sigmoid'))

# Getting the structure of our model
model.summary()

In [ ]:
feature, label = load_data()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(feature, label, test_size=0.1, shuffle=True)

x_train, x_test = tf.cast(x_train, tf.float32), tf.cast(x_test, tf.float32)

train_dataset= tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_dataset = train_dataset.batch(batch_size= 1)


In [ ]:
# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit_generator(steps_per_epoch=100, generator=training_data, verbose=1, validation_data=testing_data, epochs=10 )


In [ ]:
import matplotlib.pyplot as plt
def plot(history):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,4))
    axes[0].plot(history.history['loss'])   
    axes[0].plot(history.history['val_loss'])
    axes[0].legend(['loss','val_loss'])

    axes[1].plot(history.history['accuracy'])   
    axes[1].plot(history.history['val_accuracy'])
    axes[1].legend(['accuracy','accuracy'])

plot(history)

In [ ]:
model.save('myVggModel.h5')

In [ ]:
model = tf.keras.models.load_model('myVggModel.h5',compile=False)


prediction = model(x_test[0:9])


In [ ]:
plt.figure(figsize=(9,9))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(x_test[i])
    plt.xlabel('Pedicted:%s\n Actual: %s'%(categories[np.argmax(prediction[i])],
                                           categories[y_test[i]]))
    plt.xticks([])
plt.show()